This is the code that will be imported into our Raspberry Pi to process and send images to our computer for classification

In [10]:
# Import packages

# reload our external packages (tracker)
import importlib

import numpy as np
import cv2
import os
from skimage import data, filters
import tracker as track

importlib.reload(track)

# Directory to save training images
os.chdir("/Users/williamlee/Documents/Git Repos/Lego-Brick-Sorter/data")
print(os.listdir())

['.DS_Store', '.keep', '1.jpg', '0.jpg']


In [12]:
# Open Video
cap = cv2.VideoCapture(0)

# Create Tracker Object
tracker = track.EuclideanDistTracker()

# Create BG Subtraction Function
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

# Create Window
cv2.startWindowThread()

# FOR TRAINING:
num_img = 0

while(True):
    # print("in loop")
    # Capture video frame by frame
    ret, frame = cap.read()
    
    
    # 0: Pre-Process Image
    window_factor = 0.5   # Size factor of display window
    x_left_crop = 210     # Left side window crop
    x_right_crop = 1920   # Right side window crop
    y_top_crop = 0        # Top side window crop
    y_bottom_crop = 1080  # Bottom side window crop
    saturation_factor = 1 # Factor to saturate image by

    # Crop and adjust saturation of image
    frame = frame[y_top_crop:y_bottom_crop, x_left_crop:x_right_crop]
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)

    # Apply MOG2 to Frame
    fgmask = fgbg.apply(frame)

    # Adjust size
    frame = cv2.resize(frame, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    fgmask = cv2.resize(fgmask, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)

    
    # 1: Object Detection
    min_area = 300 # Min area of contour to detect as piece
    
    # Iterate through discovered contours and add detections
    contours, hierarchy = cv2.findContours(image=fgmask, mode=cv2.RETR_CCOMP, method=cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    image_copy = frame.copy()
    for contour in contours:
            area = cv2.contourArea(contour)
            if(area > min_area):
                x, y, w, h = cv2.boundingRect(contour)
                #cv2.putText(image_copy, "area: " + str(area),(x,y-10),0,0.5,(255, 0, 0), 2)
                detections.append([x, y, w, h])
                
                larger = w
                if(h > w):
                    larger = h
                cv2.rectangle(image_copy, (x,y), (x+larger, y+larger), (0, 255, 0), 3)
                cv2.putText(image_copy, "PIECE IS HERE" + " time:" + str(area),(x,y-10),0,0.5,(255, 0, 0), 2)
                roi = frame[y:y+larger, x:x+larger]
                print(os.listdir())
                cv2.imwrite(str(num_img) + ".jpg", roi)
                num_img += 1
            
    
    cv2.imshow('Feed with bounding boxes', image_copy)
    cv2.imshow('MOG2 Output', fgmask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

['.DS_Store', '.keep', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '2.jpg', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '4.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '4.jpg', '5.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '4.jpg', '5.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '.keep', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '8.jpg', '.keep', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '8.jpg', '9.jpg', '.keep', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '8.jpg', '9.jpg', '.keep', '10.jpg', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '8.jpg', '9.jpg', '.keep', '11.jpg', '10.jpg', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jp

['.DS_Store', '8.jpg', '9.jpg', '14.jpg', '28.jpg', '29.jpg', '15.jpg', '17.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10.jpg', '38.jpg', '21.jpg', '35.jpg', '34.jpg', '20.jpg', '36.jpg', '22.jpg', '23.jpg', '37.jpg', '33.jpg', '27.jpg', '26.jpg', '32.jpg', '18.jpg', '24.jpg', '30.jpg', '31.jpg', '25.jpg', '19.jpg', '4.jpg', '5.jpg', '7.jpg', '40.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '8.jpg', '9.jpg', '14.jpg', '28.jpg', '29.jpg', '15.jpg', '17.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10.jpg', '38.jpg', '21.jpg', '35.jpg', '34.jpg', '20.jpg', '36.jpg', '22.jpg', '23.jpg', '37.jpg', '33.jpg', '27.jpg', '26.jpg', '32.jpg', '18.jpg', '24.jpg', '30.jpg', '31.jpg', '25.jpg', '19.jpg', '4.jpg', '5.jpg', '7.jpg', '41.jpg', '40.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg', '0.jpg']
['.DS_Store', '8.jpg', '9.jpg', '14.jpg', '28.jpg', '29.jpg', '15.jpg', '17.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10

['63.jpg', '62.jpg', '60.jpg', '48.jpg', '49.jpg', '61.jpg', '59.jpg', '65.jpg', '.DS_Store', '64.jpg', '58.jpg', '8.jpg', '66.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '29.jpg', '15.jpg', '17.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10.jpg', '38.jpg', '21.jpg', '35.jpg', '34.jpg', '20.jpg', '36.jpg', '22.jpg', '23.jpg', '37.jpg', '33.jpg', '27.jpg', '26.jpg', '32.jpg', '18.jpg', '24.jpg', '30.jpg', '31.jpg', '25.jpg', '19.jpg', '42.jpg', '4.jpg', '56.jpg', '5.jpg', '57.jpg', '43.jpg', '55.jpg', '7.jpg', '41.jpg', '69.jpg', '68.jpg', '40.jpg', '54.jpg', '6.jpg', '2.jpg', '50.jpg', '44.jpg', '45.jpg', '3.jpg', '51.jpg', '47.jpg', '53.jpg', '1.jpg', '52.jpg', '0.jpg', '46.jpg']
['63.jpg', '62.jpg', '60.jpg', '48.jpg', '49.jpg', '61.jpg', '59.jpg', '65.jpg', '.DS_Store', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '66.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '29.jpg', '15.jpg', '17.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10.jpg', '38.jp

['63.jpg', '77.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jpg', '65.jpg', '.DS_Store', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '66.jpg', '72.jpg', '73.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '29.jpg', '15.jpg', '17.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10.jpg', '38.jpg', '21.jpg', '35.jpg', '34.jpg', '20.jpg', '36.jpg', '22.jpg', '23.jpg', '37.jpg', '33.jpg', '27.jpg', '26.jpg', '32.jpg', '18.jpg', '24.jpg', '30.jpg', '31.jpg', '25.jpg', '19.jpg', '42.jpg', '4.jpg', '56.jpg', '81.jpg', '80.jpg', '5.jpg', '57.jpg', '43.jpg', '55.jpg', '7.jpg', '41.jpg', '69.jpg', '82.jpg', '83.jpg', '68.jpg', '40.jpg', '54.jpg', '6.jpg', '78.jpg', '2.jpg', '50.jpg', '44.jpg', '86.jpg', '45.jpg', '3.jpg', '51.jpg', '79.jpg', '47.jpg', '53.jpg', '1.jpg', '84.jpg', '85.jpg', '52.jpg', '0.jpg', '46.jpg']
['63.jpg', '77.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jp

['63.jpg', '77.jpg', '88.jpg', '89.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jpg', '65.jpg', '.DS_Store', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '66.jpg', '72.jpg', '99.jpg', '98.jpg', '73.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '101.jpg', '100.jpg', '29.jpg', '15.jpg', '17.jpg', '102.jpg', '103.jpg', '16.jpg', '12.jpg', '.keep', '13.jpg', '39.jpg', '11.jpg', '10.jpg', '38.jpg', '21.jpg', '35.jpg', '34.jpg', '20.jpg', '36.jpg', '22.jpg', '23.jpg', '37.jpg', '33.jpg', '27.jpg', '26.jpg', '32.jpg', '18.jpg', '24.jpg', '30.jpg', '31.jpg', '25.jpg', '19.jpg', '42.jpg', '4.jpg', '56.jpg', '81.jpg', '95.jpg', '94.jpg', '80.jpg', '5.jpg', '57.jpg', '43.jpg', '55.jpg', '7.jpg', '41.jpg', '69.jpg', '96.jpg', '82.jpg', '83.jpg', '97.jpg', '68.jpg', '40.jpg', '54.jpg', '6.jpg', '78.jpg', '2.jpg', '50.jpg', '44.jpg', '93.jpg', '87.jpg', '86.jpg', '92.jpg', '45.jpg', '3.jpg', '51.jpg', '79.jpg', '47.jpg', '53.jpg', '1.jpg', '84.jpg', '9

...

['63.jpg', '189.jpg', '77.jpg', '162.jpg', '176.jpg', '88.jpg', '228.jpg', '214.jpg', '200.jpg', '201.jpg', '215.jpg', '229.jpg', '177.jpg', '89.jpg', '163.jpg', '188.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '149.jpg', '175.jpg', '161.jpg', '203.jpg', '217.jpg', '216.jpg', '202.jpg', '160.jpg', '174.jpg', '148.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jpg', '65.jpg', '170.jpg', '164.jpg', '158.jpg', '206.jpg', '212.jpg', '.DS_Store', '213.jpg', '207.jpg', '159.jpg', '165.jpg', '171.jpg', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '198.jpg', '66.jpg', '72.jpg', '167.jpg', '99.jpg', '173.jpg', '211.jpg', '205.jpg', '239.jpg', '238.jpg', '204.jpg', '210.jpg', '172.jpg', '166.jpg', '98.jpg', '73.jpg', '199.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '129.jpg', '101.jpg', '115.jpg', '277.jpg', '263.jpg', '262.jpg', '276.jpg', '114.jpg', '100.jpg', '128.jpg', '29.jpg', '15.jpg', '17.jpg', '116.jpg', '102.jpg', '248.jpg', '260.jpg', '274.jpg', '275.jpg', '261.jpg', '249.jp

['63.jpg', '189.jpg', '77.jpg', '162.jpg', '176.jpg', '88.jpg', '228.jpg', '214.jpg', '200.jpg', '201.jpg', '215.jpg', '229.jpg', '177.jpg', '89.jpg', '163.jpg', '188.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '149.jpg', '175.jpg', '161.jpg', '203.jpg', '217.jpg', '216.jpg', '202.jpg', '160.jpg', '174.jpg', '148.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jpg', '65.jpg', '170.jpg', '164.jpg', '158.jpg', '206.jpg', '212.jpg', '.DS_Store', '213.jpg', '207.jpg', '159.jpg', '165.jpg', '171.jpg', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '198.jpg', '66.jpg', '72.jpg', '167.jpg', '99.jpg', '173.jpg', '211.jpg', '205.jpg', '239.jpg', '238.jpg', '204.jpg', '210.jpg', '172.jpg', '166.jpg', '98.jpg', '73.jpg', '199.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '129.jpg', '101.jpg', '115.jpg', '288.jpg', '277.jpg', '263.jpg', '262.jpg', '276.jpg', '114.jpg', '100.jpg', '128.jpg', '29.jpg', '15.jpg', '17.jpg', '116.jpg', '102.jpg', '248.jpg', '260.jpg', '274.jpg', '275.jpg', '261.jp

['63.jpg', '189.jpg', '77.jpg', '162.jpg', '176.jpg', '88.jpg', '228.jpg', '214.jpg', '200.jpg', '201.jpg', '215.jpg', '229.jpg', '177.jpg', '89.jpg', '163.jpg', '188.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '149.jpg', '175.jpg', '161.jpg', '203.jpg', '217.jpg', '216.jpg', '202.jpg', '160.jpg', '174.jpg', '148.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jpg', '65.jpg', '170.jpg', '164.jpg', '158.jpg', '206.jpg', '212.jpg', '.DS_Store', '213.jpg', '207.jpg', '159.jpg', '165.jpg', '171.jpg', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '198.jpg', '66.jpg', '72.jpg', '167.jpg', '99.jpg', '173.jpg', '211.jpg', '205.jpg', '239.jpg', '238.jpg', '204.jpg', '210.jpg', '172.jpg', '166.jpg', '98.jpg', '73.jpg', '199.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '129.jpg', '101.jpg', '115.jpg', '288.jpg', '277.jpg', '263.jpg', '262.jpg', '276.jpg', '289.jpg', '114.jpg', '100.jpg', '128.jpg', '29.jpg', '15.jpg', '17.jpg', '116.jpg', '102.jpg', '248.jpg', '260.jpg', '274.jpg', '275.jp

['63.jpg', '189.jpg', '77.jpg', '162.jpg', '176.jpg', '88.jpg', '228.jpg', '214.jpg', '200.jpg', '201.jpg', '215.jpg', '229.jpg', '177.jpg', '89.jpg', '163.jpg', '188.jpg', '76.jpg', '62.jpg', '74.jpg', '60.jpg', '48.jpg', '149.jpg', '175.jpg', '161.jpg', '203.jpg', '217.jpg', '216.jpg', '202.jpg', '160.jpg', '174.jpg', '148.jpg', '49.jpg', '61.jpg', '75.jpg', '59.jpg', '71.jpg', '65.jpg', '170.jpg', '164.jpg', '158.jpg', '206.jpg', '212.jpg', '.DS_Store', '213.jpg', '207.jpg', '159.jpg', '165.jpg', '171.jpg', '64.jpg', '70.jpg', '58.jpg', '8.jpg', '198.jpg', '66.jpg', '72.jpg', '167.jpg', '99.jpg', '173.jpg', '211.jpg', '205.jpg', '239.jpg', '238.jpg', '204.jpg', '210.jpg', '172.jpg', '166.jpg', '98.jpg', '73.jpg', '199.jpg', '67.jpg', '9.jpg', '14.jpg', '28.jpg', '129.jpg', '101.jpg', '115.jpg', '288.jpg', '277.jpg', '263.jpg', '262.jpg', '276.jpg', '289.jpg', '114.jpg', '100.jpg', '128.jpg', '29.jpg', '15.jpg', '17.jpg', '116.jpg', '102.jpg', '248.jpg', '260.jpg', '274.jpg', '275.jp

KeyboardInterrupt: 

In [ ]:
# Open Video
cap = cv2.VideoCapture(0)

# Create Tracker Object
tracker = track.EuclideanDistTracker()

# Create BG Subtraction Function
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

# Create Window
cv2.startWindowThread()

# FOR TRAINING:
num_img = 0

while(True):
    # print("in loop")
    # Capture video frame by frame
    ret, frame = cap.read()
    
    
    # 0: Pre-Process Image
    window_factor = 0.5   # Size factor of display window
    x_left_crop = 210     # Left side window crop
    x_right_crop = 1920   # Right side window crop
    y_top_crop = 0        # Top side window crop
    y_bottom_crop = 1080  # Bottom side window crop
    saturation_factor = 1 # Factor to saturate image by

    # Crop and adjust saturation of image
    frame = frame[y_top_crop:y_bottom_crop, x_left_crop:x_right_crop]
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)

    # Apply MOG2 to Frame
    fgmask = fgbg.apply(frame)

    # Adjust size
    frame = cv2.resize(frame, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    fgmask = cv2.resize(fgmask, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)

    
    # 1: Object Detection
    min_area = 300 # Min area of contour to detect as piece
    
    # Iterate through discovered contours and add detections
    contours, hierarchy = cv2.findContours(image=fgmask, mode=cv2.RETR_CCOMP, method=cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    image_copy = frame.copy()
    for contour in contours:
            area = cv2.contourArea(contour)
            if(area > min_area):
                x, y, w, h = cv2.boundingRect(contour)
                #cv2.putText(image_copy, "area: " + str(area),(x,y-10),0,0.5,(255, 0, 0), 2)
                detections.append([x, y, w, h])
                
                larger = w
                if(h > w):
                    larger = h
                cv2.rectangle(image_copy, (x,y), (x+larger, y+larger), (0, 255, 0), 3)
                cv2.putText(image_copy, "PIECE IS HERE" + " time:" + str(area),(x,y-10),0,0.5,(255, 0, 0), 2)
                roi = frame[y:y+larger, x:x+larger]
                cv2.imwrite(str(num_img) + ".jpg", roi)

                
#     # 2: Object Tracking
#     min_age = -1 # Min age of object to classify as brick
#     interval = 1 # Frequency of images to save after being classified as brick
    
#     box_ids = tracker.update(detections)
#     for box_id in box_ids:
#         x, y, w, h, id, age = box_id
#         # If a box is "old enough" box is determined to be a piece --> take pictures
#         if age > min_age:
#             num_img += 1
#             larger = w
#             if(h > w):
#                 larger = h
#             cv2.rectangle(image_copy, (x,y), (x+larger, y+larger), (0, 255, 0), 3)
#             cv2.putText(image_copy, "PIECE IS HERE" + " time:" + str(age),(x,y-10),0,0.5,(255, 0, 0), 2)
            
#             # Save bounding box as image every X frames
#             roi = frame[y:y+larger, x:x+larger]
#             #print(str(num_img) + ".jpg")
#             true_age = age - min_age
#             if(true_age % interval == 0):
#                 print(true_age)
#                 print("save image")
#                 #cv2.imwrite(str(num_img) + ".jpg", roi)
            
    
    cv2.imshow('Feed with bounding boxes', image_copy)
    cv2.imshow('MOG2 Output', fgmask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

In [13]:
# Open Video
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 3) # auto mode
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1) # manual mode
cap.set(cv2.CAP_PROP_EXPOSURE, 50)
# Create Window
cv2.startWindowThread()

# FOR TRAINING:
num_img = 0

while(True):
    # print("in loop")
    # Capture video frame by frame
    ret, frame = cap.read()
    cv2.imshow('MOG2 Output', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.7.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
